Reference : https://www.machinelearningplus.com/time-series/vector-autoregression-examples-python/

In [1]:
import pandas as pd 
import numpy as np 
from statsmodels.tsa.stattools import grangercausalitytests

In [29]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
train['datetime'] = pd.to_datetime(train['datetime'])
train.set_index('datetime', inplace=True)

In [30]:
ts_cols = ['temperature', 'var1', 'pressure', 'windspeed', 'electricity_consumption']

In [31]:
train[ts_cols]

,temperature,var1,pressure,windspeed,electricity_consumption
datetime,,,,,
2013-07-01 00:00:00,-11.4,-17.1,1003.0,571.910,216.0
2013-07-01 01:00:00,-12.1,-19.3,996.0,575.040,210.0
2013-07-01 02:00:00,-12.9,-20.0,1000.0,578.435,225.0
2013-07-01 03:00:00,-11.4,-17.1,995.0,582.580,216.0
2013-07-01 04:00:00,-11.4,-19.3,1005.0,586.600,222.0
...,...,...,...,...,...
2017-06-23 19:00:00,-0.7,-15.0,1009.0,51.685,225.0
2017-06-23 20:00:00,-2.9,-11.4,1005.0,56.105,213.0
2017-06-23 21:00:00,-1.4,-12.9,995.0,61.275,213.0


In [3]:
train.head()

,ID,temperature,var1,pressure,windspeed,var2,electricity_consumption
datetime,,,,,,,
2013-07-01 00:00:00,0,-11.4,-17.1,1003.0,571.910,A,216.0
2013-07-01 01:00:00,1,-12.1,-19.3,996.0,575.040,A,210.0
2013-07-01 02:00:00,2,-12.9,-20.0,1000.0,578.435,A,225.0
2013-07-01 03:00:00,3,-11.4,-17.1,995.0,582.580,A,216.0
2013-07-01 04:00:00,4,-11.4,-19.3,1005.0,586.600,A,222.0


# Testing Causation using Granger’s Causality Test

In [5]:
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [6]:
grangers_causation_matrix(train, variables = ts_cols)    

e:\python36\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
e:\python36\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
e:\python36\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
e:\python36\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
e:\python36\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constra

,temperature_x,var1_x,pressure_x,windspeed_x,electricity_consumption_x
temperature_y,1.0,0.0,0.0000,0.0,0.0000
var1_y,0.0,1.0,0.0000,0.0,0.0000
pressure_y,0.0,0.0,1.0000,0.0,0.0004
windspeed_y,0.0,0.0,0.0011,1.0,0.0000
electricity_consumption_y,0.0,0.0,0.0000,0.0,1.0000


# Cointegration Test

In [7]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

In [8]:
cointegration_test(train[ts_cols])

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
temperature ::  3796.01   > 60.0627   =>   True
var1   ::  1508.18   > 40.1749   =>   True
pressure ::  645.39    > 24.2761   =>   True
windspeed ::  40.31     > 12.3212   =>   True
electricity_consumption ::  0.02      > 4.1296    =>   False


# Testing Stationarity

In [9]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [10]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [11]:
# ADF Test on each column
for name, column in train[ts_cols].iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "temperature" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.8642
 No. Lags Chosen       = 49
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0497. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "var1" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.9272
 No. Lags Chosen       = 48
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "pressure" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance 

# Fitting VAR

In [12]:
from scipy.special import logsumexp

In [16]:
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse, aic

In [17]:
model = VAR(train[ts_cols])

e:\python36\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [20]:
for i in [10, 20 ,30 ,40, 50, 60, 70, 80, 90, 100]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 10
AIC :  18.028603158908314
BIC :  18.107400061063196
FPE :  67565170.75399905
HQIC:  18.054036811276116 

Lag Order = 20
AIC :  17.88151404406285
BIC :  18.037614547927692
FPE :  58323416.686788335
HQIC:  17.931900259465053 

Lag Order = 30
AIC :  17.84698757687258
BIC :  18.080442961435654
FPE :  56344105.9580498
HQIC:  17.922343809705616 

Lag Order = 40
AIC :  17.8503598969047
BIC :  18.16122149525422
FPE :  56534484.70987542
HQIC:  17.95070362048442 

Lag Order = 50
AIC :  17.839308863397164
BIC :  18.22762806279902
FPE :  55913237.30423222
HQIC:  17.964657569986507 

Lag Order = 60
AIC :  17.84359285835442
BIC :  18.309421100330074
FPE :  56153399.41726604
HQIC:  17.99396405919117 

Lag Order = 70
AIC :  17.845251916615933
BIC :  18.388640697020445
FPE :  56246801.118532516
HQIC:  18.02066314194051 

Lag Order = 80
AIC :  17.846351897482723
BIC :  18.467352766582913
FPE :  56308922.360256255
HQIC:  18.04682069656601 

Lag Order = 90
AIC :  17.851042937201658
BIC :  1

Choosing lag of 10

In [22]:
result = model.fit(10)

In [23]:
result.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 03, May, 2020
Time:                     23:15:54
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                    18.1074
Nobs:                     26486.0    HQIC:                   18.0540
Log likelihood:          -426408.    FPE:                6.75652e+07
AIC:                      18.0286    Det(Omega_mle):     6.69184e+07
--------------------------------------------------------------------
Results for equation temperature
                                 coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------------------
const                              25.786712         1.667203           15.467           0.000
L1.temperature                      0.789290         0.006148          128.379           0.00

In [24]:
result.k_ar

10

In [26]:
forecast_input = train[ts_cols].values[-result.k_ar:]
forecast_input

array([[ 7.0000e-01, -1.4300e+01,  1.0110e+03,  1.8630e+01,  2.0400e+02],
       [ 7.0000e-01, -1.2900e+01,  9.9500e+02,  2.6655e+01,  2.0100e+02],
       [-7.0000e-01, -1.5000e+01,  1.0010e+03,  3.2340e+01,  2.0100e+02],
       [-7.0000e-01, -1.2900e+01,  9.9500e+02,  3.8865e+01,  2.2200e+02],
       [-2.1000e+00, -1.2900e+01,  1.0020e+03,  4.6890e+01,  2.1600e+02],
       [-7.0000e-01, -1.5000e+01,  1.0090e+03,  5.1685e+01,  2.2500e+02],
       [-2.9000e+00, -1.1400e+01,  1.0050e+03,  5.6105e+01,  2.1300e+02],
       [-1.4000e+00, -1.2900e+01,  9.9500e+02,  6.1275e+01,  2.1300e+02],
       [-2.9000e+00, -1.1400e+01,  9.9600e+02,  6.7210e+01,  2.1000e+02],
       [-2.1000e+00, -1.1400e+01,  1.0090e+03,  7.1880e+01,  2.1000e+02]])

In [27]:
fc = result.forecast(y=forecast_input, steps=10)

In [28]:
df_forecast = pd.DataFrame(fc, index=train[ts_cols].index[-10:], columns=train[ts_cols].columns + '_2d')
df_forecast

,temperature_2d,var1_2d,pressure_2d,windspeed_2d,electricity_consumption_2d
datetime,,,,,
2017-06-23 14:00:00,-2.467144,-11.741235,1000.973461,69.307850,215.541436
2017-06-23 15:00:00,-2.515684,-11.707709,1000.385688,66.297676,218.671955
2017-06-23 16:00:00,-2.628436,-11.631713,1000.596825,64.045190,222.775356
2017-06-23 17:00:00,-2.737850,-11.611856,1000.565083,61.318165,226.323570
2017-06-23 18:00:00,-2.844533,-11.533644,1000.671236,58.831751,232.115008
2017-06-23 19:00:00,-2.897776,-11.532838,1000.501413,56.868347,236.599978
2017-06-23 20:00:00,-2.992083,-11.411232,999.983352,54.902452,240.582401
2017-06-23 21:00:00,-2.991196,-11.351502,999.716114,53.232043,243.212787
2017-06-23 22:00:00,-3.004961,-11.260114,999.984403,51.702385,246.478589
